In [7]:
%pip install autoflake


  Obtaining dependency information for autoflake from https://files.pythonhosted.org/packages/9e/a5/8471753bc95672fb16d9cd1cb82ba460c66721378dd8cc8629d86c148a09/autoflake-2.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyflakes>=3.0.0 from https://files.pythonhosted.org/packages/00/e9/1e1fd7fae559bfd07704991e9a59dd1349b72423c904256c073ce88a9940/pyflakes-3.1.0-py2.py3-none-any.whl.metadata
  Using cached pyflakes-3.1.0-py2.py3-none-any.whl.metadata (3.5 kB)
Using cached pyflakes-3.1.0-py2.py3-none-any.whl (62 kB)


## Have code lots of unused imports, and want to remove them quickly? Use Autoflake!

Autoflake uses [Flake8](https://flake8.pycqa.org/en/latest/) to do some very simple cleanup on the code.  It also has an autoflake vscode extension, to run on single files with a couple shortcut keys.

In [55]:
from pathlib import Path

In [56]:
code = """
import itertools as it
import pandas as pd
from pprint import pprint

pprint('Hello, World!')
"""
path = Path('./src/main.py')
path.parent.mkdir(parents=True, exist_ok=True)
path.write_text(code)
print(path.read_text())


import itertools as it
import pandas as pd
from pprint import pprint

pprint('Hello, World!')



#### What will happen when we run autoflake on this file?  By default, it shows a diff:

In [57]:
!python -m autoflake --remove-all-unused-imports ./src/main.py

--- original/./src/main.py
+++ fixed/./src/main.py
@@ -1,6 +1,4 @@
 

-import itertools as it

-import pandas as pd

 from pprint import pprint

 

 pprint('Hello, World!')



But the original file stays the same:

In [58]:
print(path.read_text())


import itertools as it
import pandas as pd
from pprint import pprint

pprint('Hello, World!')



### To have it apply the change "in-place", use the "--in-place" or "-i" flag

In [59]:
!python -m autoflake --remove-all-unused-imports -i ./src/main.py
print(path.read_text())


from pprint import pprint

pprint('Hello, World!')



#### To do this recursively on all files, use the "-r" flag.
(no demo shown)

### Autoflake can also remove unused variables (from inside functions)!

In [60]:
code = """
# Won't be affected
a = 3
b = 4
c = 5
d = a + b

# Will be affected
def add(a, b):
   d = 10
   return a + 4


"""
path = Path('./src/main.py')
path.parent.mkdir(parents=True, exist_ok=True)
path.write_text(code)
print(path.read_text())


# Won't be affected
a = 3
b = 4
c = 5
d = a + b

# Will be affected
def add(a, b):
   d = 10
   return a + 4





In [61]:
!python -m autoflake --remove-unused-variables -i src/main.py
print(path.read_text())


# Won't be affected
a = 3
b = 4
c = 5
d = a + b

# Will be affected
def add(a, b):
   return a + 4





### Feeling lazy about imports? It can also expand star imports!

In [62]:
code = """
from math import *

sqrt(32)
sin(6)

"""
path = Path('./src/main.py')
path.parent.mkdir(parents=True, exist_ok=True)
path.write_text(code)
print(path.read_text())


from math import *

sqrt(32)
sin(6)




In [63]:
!python -m autoflake --expand-star-imports -i src/main.py
print(path.read_text())


from math import sin, sqrt

sqrt(32)
sin(6)




Note that autoflake doesn't guess what to do if there are multiple start imports in the same file:

In [64]:
code = """
from math import *
from pprint import *
from itertools import *

sqrt(32)
sin(6)
pprint('hi!')

"""
path = Path('./src/main.py')
path.parent.mkdir(parents=True, exist_ok=True)
path.write_text(code)
print(path.read_text())


from math import *
from pprint import *
from itertools import *

sqrt(32)
sin(6)
pprint('hi!')




In [65]:
!python -m autoflake --expand-star-imports -i src/main.py
print(path.read_text())


from math import *
from pprint import *
from itertools import *

sqrt(32)
sin(6)
pprint('hi!')




Neat!  Here are the docs:

In [66]:
!python -m autoflake -h

usage: autoflake [-h] [-c | -cd]
                 [--imports IMPORTS | --remove-all-unused-imports] [-r] [-j n]
                 [--exclude globs] [--expand-star-imports]
                 [--ignore-init-module-imports] [--remove-duplicate-keys]
                 [--remove-unused-variables]
                 [--remove-rhs-for-unused-variables]
                 [--ignore-pass-statements] [--ignore-pass-after-docstring]
                 [--version] [--quiet] [-v]
                 [--stdin-display-name STDIN_DISPLAY_NAME]
                 [--config CONFIG_FILE] [-i | -s]
                 files [files ...]

Removes unused imports and unused variables as reported by pyflakes.

positional arguments:
  files                 files to format

options:
  -h, --help            show this help message and exit
  -c, --check           return error code if changes are needed
  -cd, --check-diff     return error code if changes are needed, also display
                        file diffs
  --imports IMPOR